In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind, mannwhitneyu 
from wordcloud import WordCloud
from PIL import Image
import nltk
nltk.download('stopwords')
import spacy
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
import numpy as np

In [ ]:
route = "./data/Bronze" 

elements = os.listdir(route)


files_mercadona = [element for element in elements if element.startswith("mercadona") and os.path.isfile(os.path.join(route, element))]
files_hacendado = [element for element in elements if element.startswith("hacendado") and os.path.isfile(os.path.join(route, element))]



print("Number of elementes:", len(elements))
print("Number of Mercadona files:", len(files_mercadona))
print("Number of hacendado files:", len(files_hacendado))

In [ ]:
#empezamos por los mensajes que contienen la palabra mercadona

mercadona_merged = pd.DataFrame()

mercadona_day = []
counts_tweets_mercadona = [] 

for i in files_mercadona:
    
    #dia
    b = i.strip("mercadona dia ")
    day = b.strip(".xlsx")
    mercadona_day.append(day)

    df = pd.read_excel('./data/Bronze/'+i)

    #guardar len de archivo
    lenght_file = len(df)
    counts_tweets_mercadona.append(lenght_file)

    #unir df
    mercadona_merged = pd.concat([mercadona_merged, df], ignore_index=True)


mercadona_merged


mercadona_longitud = len(mercadona_merged)

In [ ]:
mercadona_counts = pd.DataFrame({
    'tweets count': counts_tweets_mercadona,
    'Day': pd.to_datetime(mercadona_day),
    'Type': 'mercadona'
})

plt.figure(figsize=(15,6)) 
plt.bar(mercadona_counts['Day'],mercadona_counts['tweets count'])
plt.title("Nº tweets Mercadona por día")
plt.show()

print("Numero tweets mercadona: ", len(mercadona_merged))
merca_days_Zero_TW = len(mercadona_counts[mercadona_counts['tweets count'] == 0])
print('Numero de archvios con count  0: ', merca_days_Zero_TW)




In [ ]:
num_duplicates = mercadona_counts.duplicated().sum()
print(f"Numero duplicados: {num_duplicates}")


duplicados_unicos = mercadona_merged['tweets'][mercadona_merged['tweets'].duplicated()].unique()
print("Numero Tweets duplicados: ", len(duplicados_unicos))

tweets_duplicados = mercadona_merged['tweets'].str.lower().value_counts() # duplicates in lower case
tweets_duplicados = tweets_duplicados[tweets_duplicados > 1]
df_tweets_duplicados = pd.DataFrame(tweets_duplicados)
print("number of duplicates in lowercase: ", len(df_tweets_duplicados))
df_tweets_duplicados.head(20)

In [ ]:
# Se observa que hay varios mensajes con alta frecuencia.
# Se verificará si provienen de los mismos usuarios o de usuarios diferentes.
# Si son de los mismos usuarios, se eliminan.
# Además, notamos que algunos usuarios tienen "mercadona" en sus nombres de usuario, 
# por lo que mantendremos solo los mensajes que mencionen explícitamente "mercadona" y eliminaremos el resto.

mercadona_merged.describe()

In [ ]:
##Bot MasterHdeRepollo
print(mercadona_merged[mercadona_merged['tweets'].str.lower()=="me voy a la inaguracion del mercadona de mi pueblo!!"]['User'].value_counts())
print(mercadona_merged[mercadona_merged['User']=="MasterHdeRepollo"]['tweets'].value_counts())

In [ ]:
mercadona_merged[mercadona_merged['tweets'].str.lower()=="polla sabor helado de nubes del mercadona"]['User'].value_counts()

In [ ]:
mercadona_merged[mercadona_merged['tweets'].str.lower()=="coño sabor helado de nubes del mercadona"]['User'].value_counts()

In [ ]:
## Bot genitalesabores_bot
print(mercadona_merged[mercadona_merged['User']=="genitalesabores_bot"]['tweets'].value_counts())

In [ ]:
mercadona_merged[mercadona_merged['tweets'].str.lower()=="mercadona"]['User'].value_counts()

In [ ]:
mercadona_merged[mercadona_merged['tweets'].str.lower()=="quiero chupachups azules y zumitos del mercadona"]['User'].value_counts()

In [ ]:
print(mercadona_merged[mercadona_merged['User']=="ansufatisima (roja no, rojisima)"]['tweets'].value_counts())
#no parece un bot

In [ ]:
mercadona_merged[mercadona_merged['tweets'].str.lower()=="el día que suban de precio el vodka knebep de 4 euros de mercadona se lía la de dios."]['User'].value_counts()

In [ ]:
mercadona_merged[mercadona_merged['tweets'].str.lower()=="mucho polito ralph lauren y mucha camisa tommy hilfiger pero para el botellón compráis ron de 5 euros del mercadona."]['User'].value_counts()

In [ ]:
### Este usuario tiene 17486 mensajes pero solamente 7 hablan de mercadona. se eliminam los tweets que no contengan mercadona

print(mercadona_merged[mercadona_merged['User']=="pasillo de mercadona"]['tweets'].value_counts())


mercadona_merged[
    (mercadona_merged['User'] == "pasillo de mercadona") &
    (mercadona_merged['tweets'].str.lower().str.contains("mercadona"))
]

In [ ]:
print("Tweets Number: ", len(mercadona_merged))

print("Tweets Number that contain mercadona in tweets colum: ",  len(mercadona_merged[mercadona_merged['tweets'].str.lower().str.contains("mercadona")]))

In [ ]:
mercadona_merged_clean = mercadona_merged[mercadona_merged['User'] != "genitalesabores_bot"]
mercadona_merged_clean = mercadona_merged_clean[mercadona_merged_clean ['User'] != "MasterHdeRepollo"]
mercadona_merged_clean = mercadona_merged_clean[mercadona_merged_clean['tweets'].str.lower().str.contains("mercadona")]
mercadona_drop_tw = (len(mercadona_merged) - len(mercadona_merged_clean))
print("Tweets cleaned = ", mercadona_drop_tw)

In [ ]:
## por algun motivo no se ha guardado el nombre de algunos usuarios, seguramente por contener caracteres extraños.
print(mercadona_merged_clean.isna().sum())
mercadona_nan = len(mercadona_merged_clean[mercadona_merged_clean['User'].isna()])
mercadona_merged_clean[mercadona_merged_clean['User'].isna()]


In [ ]:
mercadona_merged_clean[mercadona_merged_clean['User']=="USER_EMPTY"]

In [ ]:
### Imputar "USER_EMPTY" en usuarios NaN. Aunque en el analisis no se utilizarán los nombres por protección de datos.
mercadona_merged_clean['User'] = mercadona_merged_clean['User'].fillna("USER_EMPTY")
mercadona_merged_clean[mercadona_merged_clean['User']=="USER_EMPTY"]

In [ ]:
len(mercadona_merged_clean)

In [ ]:
mercadona_merged_clean = mercadona_merged_clean[~mercadona_merged_clean['tweets'].str.replace(r'\s+', ' ', regex=True)
                      .str.strip()
                      .str.lower()
                      .duplicated()]
mercadona_merged_clean

In [ ]:
#comprobamos
duplicados_unicos = mercadona_merged_clean['tweets'][mercadona_merged_clean['tweets'].duplicated()].unique()
print("Number of duplicates Tweets: ", len(duplicados_unicos))

tweets_duplicados = mercadona_merged_clean['tweets'].str.lower().value_counts() # duplicates in lower case
tweets_duplicados = tweets_duplicados[tweets_duplicados > 1]
df_tweets_duplicados = pd.DataFrame(tweets_duplicados)
print("number of duplicates in lowercase: ", len(df_tweets_duplicados))
df_tweets_duplicados.head(20)

In [ ]:
#es el turno de hacendado

hacendado_merged = pd.DataFrame()

hacendado_day = []
counts_tweets_hacendado = [] 

for i in files_hacendado:
    
    #save file day
    b = i.strip("hacendado dia ")
    day = b.strip(".xlsx")
    hacendado_day.append(day)

    df = pd.read_excel('./data/Bronze/'+i)

    #save file lenght in a list
    lenght_file = len(df)
    counts_tweets_hacendado.append(lenght_file)

    #merge all files in a single df
    hacendado_merged = pd.concat([hacendado_merged, df], ignore_index=True)



hacendado_longitud = len(hacendado_merged)

hacendado_merged


In [ ]:
hacendado_counts = pd.DataFrame({
    'tweets count': counts_tweets_hacendado,
    'Day': pd.to_datetime(hacendado_day),
    'Type': 'hacendado'
})
plt.figure(figsize=(15,6)) 
plt.bar(hacendado_counts['Day'],hacendado_counts['tweets count'])
plt.title("Nº tweets hacendado por día")
plt.show()

print("Number of hacendado tweets: ", len(hacendado_merged))
print('Number of days with a tweet count of 0: ', len(hacendado_counts[hacendado_counts['tweets count'] == 0]))


In [ ]:
num_duplicates = hacendado_counts.duplicated().sum()
print(f"Number of duplicates: {num_duplicates}")


duplicados_unicos = hacendado_merged['tweets'][hacendado_merged['tweets'].duplicated()].unique()
print("Number of duplicates Tweets: ", len(duplicados_unicos))

tweets_duplicados = hacendado_merged['tweets'].str.lower().value_counts() # duplicates in lower case
tweets_duplicados = tweets_duplicados[tweets_duplicados > 1]
df_tweets_duplicados = pd.DataFrame(tweets_duplicados)
print("number of duplicates in lowercase: ", len(df_tweets_duplicados))
df_tweets_duplicados.head(20)

In [ ]:
hacendado_merged['User'].value_counts().head(10)

In [ ]:
# print(hacendado_merged[hacendado_merged['User']=="DARIAK"]['tweets'].value_counts())


# hacendado_merged[
#     (hacendado_merged['User'] == "DARIAK") &
#     (hacendado_merged['tweets'].str.lower().str.contains("hacendado"))
# ]

In [ ]:
#mercadona_hacendado[mercadona_hacendado['tweets']=="hacendado"]  poner para borrar

In [ ]:
hacendado_merged_clean =  hacendado_merged[hacendado_merged['tweets'].str.lower().str.contains("hacendado")]
hacendado_merged_clean = hacendado_merged_clean[~hacendado_merged_clean['tweets'].str.replace(r'\s+', ' ', regex=True)
                      .str.strip()
                      .str.lower()
                      .duplicated()]


hacendado_drop_tw = (len(hacendado_merged) - len(hacendado_merged_clean))
print("Tweets cleaned = ", hacendado_drop_tw)

In [ ]:
hacendado_merged_clean['User'].value_counts().head(10)

In [ ]:
#se detectan estos bots
# Infladona              
# Precio carrito de la compra 
print(len(hacendado_merged_clean))
hacendado_merged_clean = hacendado_merged_clean[hacendado_merged_clean ['User'] != "Infladona"]
hacendado_merged_clean = hacendado_merged_clean[hacendado_merged_clean ['User'] != "Precio carrito de la compra "]
print(len(hacendado_merged_clean))

In [ ]:
# for i in hacendado_merged_clean['tweets'][hacendado_merged_clean ['User'] == "."]:
#     print("====")
#     print(i)

In [ ]:
print(hacendado_merged_clean.isna().sum())
hacendado_nan = len(hacendado_merged_clean[hacendado_merged_clean['User'].isna()])
hacendado_merged_clean[hacendado_merged_clean['User'].isna()]


In [ ]:
hacendado_merged_clean['User'] = hacendado_merged_clean['User'].fillna("USER_EMPTY")
hacendado_merged_clean[hacendado_merged_clean['User']=="USER_EMPTY"]

In [ ]:
hacendado_merged_clean[hacendado_merged_clean['User'].isna()]

In [ ]:
densidad tweets día
sns.kdeplot(data=mercadona_counts, x='tweets count', hue='Type', fill=True, common_norm=True) 
plt.title("Histograma nº tweets/día")
plt.show()

sns.kdeplot(data=hacendado_counts, x='tweets count', hue='Type', fill=True, common_norm=True) 
plt.title("Hacendado nº tweets/día")
plt.show()

In [ ]:
#### 

print(f"""Resumen de la búsqueda:

========== Mercadona =========

Se recopiló {len(files_mercadona)} archivos, un archivo por día durante diez años — de los cuales solo {merca_days_Zero_TW} días no tuvieron tweets.

En total, se obtuvieron {len(mercadona_merged)} tweets en este período de diez años.

Se eliminó {mercadona_drop_tw} mensajes correspondientes a bots o tweets que no contenían la palabra clave "mercadona",
lo que resultó en un conteo final de {len(mercadona_merged_clean)} tweets.

Además, se imputó {mercadona_nan} valores de usuario faltantes (NaN) con "USER_EMPTY".


========== hacendado =========

Se recopiló {len(files_hacendado)} archivos, un archivo por día durante diez años — de los cuales solo {merca_days_Zero_TW} días no tuvieron tweets.

En total, se obtuvieron {len(hacendado_merged)} tweets en este período de diez años.

Se eliminó {hacendado_drop_tw} mensajes correspondientes a bots o tweets que no contenían la palabra clave "hacendado",
lo que resultó en un conteo final de {len(hacendado_merged_clean)} tweets.

Además, se imputó {hacendado_nan} valores de usuario faltantes (NaN) con "USER_EMPTY".
""")


In [ ]:
#unimos los dos dataframes
mercadona_hacendado = pd.concat([mercadona_merged_clean, hacendado_merged_clean], axis = 0)

In [ ]:
#comprobar los duplciados en columnas fechaq tweet y usuario. los que tienen mercadona y hacedando
duplicados = mercadona_hacendado.duplicated(subset=['User', 'tweets', 'fecha_captura'], keep=False)
filas_duplicadas = mercadona_hacendado[duplicados]
filas_duplicadas

In [ ]:
print(len(mercadona_hacendado))
mercadona_hacendado.drop_duplicates(subset=['User', 'tweets', 'fecha_captura'], inplace=True)
print(len(mercadona_hacendado))

In [ ]:

def len_tweets_plot(mercadona_hacendado, nombre: str):
    # Calcular longitud de texto
    mercadona_hacendado["char_len"] = mercadona_hacendado["tweets"].apply(lambda x: len(x))

    mercadona = mercadona_hacendado[mercadona_hacendado['search'] == 'mercadona']['char_len']
    hacendado = mercadona_hacendado[mercadona_hacendado['search'] == 'hacendado']['char_len']
    #cálculo medias y desviación
    media_mercadona = mercadona.mean()
    media_hacendado = hacendado.mean()
    std_mercadona = mercadona.std()
    std_hacendado = hacendado.std()

    # Crear gráfico de distribución
    plt.figure(figsize=(10, 6))
    sns.set_style("darkgrid")
    sns.kdeplot(data=mercadona_hacendado, x="char_len", hue="search", fill=True, common_norm=True) #common_norm=True
    # Añadir líneas de media
    plt.axvline(media_mercadona, color='blue', linestyle='--', linewidth=1, label='Media mercadonas')
    plt.axvline(media_hacendado, color="orange", linestyle='--', linewidth=1, label='Media hacendado')
    plt.text(media_mercadona + 2, plt.ylim()[1]*0.8, f"Media mercadona: {int(media_mercadona)}, std:{int(std_mercadona)} ", color='blue', fontsize=10)
    plt.text(media_hacendado + 2, plt.ylim()[1]*0.6, f"Media hacendado: {int(media_hacendado)}, std:{int(std_hacendado)}", color='orange', fontsize=10)

    plt.xlabel('Número de caracteres', fontsize=16)
    plt.ylabel('Densidad', fontsize=16)
    plt.title(f'Distribución de longitud de texto en {nombre}', fontsize=18)
    plt.show();

    #test diferencia de medias.
    t_stat, p_value = ttest_ind(mercadona, hacendado,  equal_var=False)
    if p_value<0.05:
      print(f"La diferencia de medias ES estadísticamente significativa, p_value:{p_value:.4f}")
    else:
       print(f" La diferencia de medias NO es estadísticamente significativa, p_value:{p_value:.4f}")
    print(100*("="))




len_tweets_plot(mercadona_hacendado, "Hacendatos")
print(100*("="))
print(100*("="))
print(100*("="))


In [ ]:
# Se observa que hay algunos tweets con un numero de caracteres excesivos.  y con contenido que podría confundir a los modelos del lenguaje.
# por tanto se procede a buscar un punto de corte adecuado. 

In [ ]:
# for i in mercadona_hacendado['tweets'][mercadona_hacendado['char_len'] > 300]:
#     print("===")
#     print(i)

In [ ]:
mercadona_hacendado.sort_values(by='char_len', ascending=False)

In [ ]:
plt.hist(mercadona_hacendado['char_len'][mercadona_hacendado['char_len'] < 350], bins=10, edgecolor='black') 

In [ ]:
plt.hist(mercadona_hacendado['char_len'][(mercadona_hacendado['char_len'] > 280) & (mercadona_hacendado['char_len'] < 350)], bins=10, edgecolor='black') 

In [ ]:
mercadona_hacendado['char_len'][mercadona_hacendado['char_len'] > 280].count()
## son valores atipicos.

In [ ]:

### bsucamos outlier con z-score

mercadona_hacendado['z_score'] = (mercadona_hacendado['char_len']- mercadona_hacendado['char_len'].mean()) / mercadona_hacendado['char_len'].std()
outliers_mask = np.abs(mercadona_hacendado['z_score']) > 3

outliers = mercadona_hacendado[outliers_mask]
outliers.sort_values(by='char_len')

In [ ]:

mercadona_hacendado['z_score_por_grupo'] = mercadona_hacendado.groupby('search')['char_len'].transform(
    lambda x: (x - x.mean()) / x.std()
)
# Ahora puedes identificar los outliers
outliers_mask = np.abs(mercadona_hacendado['z_score_por_grupo']) > 3

# Filtra y muestra los outliers
outliers = mercadona_hacendado[outliers_mask]
outliers_sorted_grouped = outliers.sort_values(by='char_len', ascending=True).groupby('search')
for name, group in outliers_sorted_grouped:
    print(f"\nGrupo: {name}")
    print(group.iloc[0])

In [ ]:
#rango inter quartilico. 
Q1 = mercadona_hacendado['char_len'].quantile(0.25)
Q3 = mercadona_hacendado['char_len'].quantile(0.75)
IQR = Q3 - Q1
print(f"Q1: {Q1}")
print(f"Q3: {Q3}")
print(f"IQR: {IQR}")

## capturo mejor los outliers con z_score por grupo. 

In [ ]:
# se determina un puto de corte mediante el z_score popr grupo, ya que no sabemos si existen diferencias entre ellos.  
a = len(mercadona_hacendado)
print(len(mercadona_hacendado))
mercadona_hacendado = mercadona_hacendado[np.abs(mercadona_hacendado['z_score_por_grupo']) < 3]
print(len(mercadona_hacendado))
b= len(mercadona_hacendado)
print(a-b) #capturo 3 outlier más que solo por grupo. 

In [ ]:
def len_tweets_plot(mercadona_hacendado, nombre: str):
    # Calcular longitud de texto
    mercadona_hacendado["char_len"] = mercadona_hacendado["tweets"].apply(lambda x: len(x))

    mercadona = mercadona_hacendado[mercadona_hacendado['search'] == 'mercadona']['char_len']
    hacendado = mercadona_hacendado[mercadona_hacendado['search'] == 'hacendado']['char_len']
    #cálculo medias y desviación
    media_mercadona = mercadona.mean()
    media_hacendado = hacendado.mean()
    std_mercadona = mercadona.std()
    std_hacendado = hacendado.std()

    # Crear gráfico de distribución
    plt.figure(figsize=(10, 6))
    sns.set_style("darkgrid")
    sns.kdeplot(data=mercadona_hacendado, x="char_len", hue="search", fill=True, common_norm=False) #common_norm=True
    # Añadir líneas de media
    plt.axvline(media_mercadona, color='blue', linestyle='--', linewidth=1, label='Media mercadonas')
    plt.axvline(media_hacendado, color="orange", linestyle='--', linewidth=1, label='Media hacendado')
    plt.text(media_mercadona + 2, plt.ylim()[1]*0.8, f"Media mercadona: {int(media_mercadona)}, std:{int(std_mercadona)} ", color='blue', fontsize=10)
    plt.text(media_hacendado + 2, plt.ylim()[1]*0.6, f"Media hacendado: {int(media_hacendado)}, std:{int(std_hacendado)}", color='orange', fontsize=10)

    plt.xlabel('Número de caracteres', fontsize=16)
    plt.ylabel('Densidad', fontsize=16)
    plt.title(f'Distribución de longitud de texto en {nombre}', fontsize=18)
    plt.show();

    #test diferencia de medias.
    t_stat, p_value = ttest_ind(mercadona, hacendado,  equal_var=False)  # equal_var=False Welch’s t-test comparar medias sin asumir varianzas iguales. con muchos datos
    #t_stat, p_value = mannwhitneyu(mercadona, hacendado) 
    if p_value<0.05:
      print(f"La diferencia de medias ES estadísticamente significativa, p_value:{p_value:.4f}")
    else:
       print(f" La diferencia de medias NO es estadísticamente significativa, p_value:{p_value:.4f}")
    print(100*("="))




len_tweets_plot(mercadona_hacendado, "Hacendatos")
print(100*("="))
print(100*("="))
print(100*("="))


In [ ]:
print('Tweets procesados: ')
print(mercadona_longitud + hacendado_longitud)
print('Numero de items limpiados: ')
print(mercadona_longitud + hacendado_longitud - len(mercadona_hacendado))
print('Numero de tweets a la siguiente fase: ')
print(len(mercadona_hacendado))

In [ ]:
# Tweets procesados: 
# 408784
# Numero de items limpiados: 
# 53270
# Numero de tweets a la siguiente fase: 
# 355514

In [ ]:
mercadona_hacendado.columns

In [ ]:
#guardado para procesado de LOPD
mercadona_hacendado = mercadona_hacendado[['User', 'tweets', 'search', 'fecha_captura']]
mercadona_hacendado.to_parquet('./data/Silver/Cleaned_data.parquet' ,index=False)

In [ ]:
mercadona_hacendado = pd.read_parquet( './data/Silver/Cleaned_data.parquet')

In [ ]:
 #modelo mediano español spicy
nlp = spacy.load('es_core_news_md', disable=['parser', 'ner'])
#stop_words en español
es_stopwords = stopwords.words('spanish') # 'english'
# iniciar Tokenizador
tweet_tokenizer = TweetTokenizer()


# Eliminar espacios
def eliminar_espacios(text):
    return  " ".join(text.split())

# To lower
def texto_to_lower(text):
  return text.lower()

def normalizar_tokens(documento):
    # Dividir el documento en palabras
    palabras =  tweet_tokenizer.tokenize(documento) # tokenize(documento)

    # Reemplazar URLs, menciones de Twitter y números por los tokens correspondientes
    for i in range(len(palabras)):
        if palabras[i].startswith("http://") or palabras[i].startswith("https://") or palabras[i].startswith("www."):
            palabras[i] = "URL"
        elif palabras[i].startswith("@"): #tokenize(documento)
            palabras[i] = "MENTION"
        #elif palabras[i].isdigit():
        elif palabras[i].isdigit(): #
            palabras[i] = "NUM"
    # Unir las palabras de nuevo en un documento modificado
    palabras = set(palabras) # para que no aparezcan cosas como MENTION MENTION,  NUM NUM, URL URL
    documento_modificado = ' '.join(palabras)


    return documento_modificado


def lematizar_eliminacion_tokens(texto):
    # Procesar el texto con el objeto nlp
    doc = nlp(texto)
    # Lematizar el texto
    lemas = [token.lemma_ for token in doc]
    # Eliminar símbolos de puntuación y stopwords
    tokens_filtrados = [token for token in lemas if token.isalpha() and token.lower() not in es_stopwords]
    # Unir los tokens filtrados en un nuevo texto
    texto_procesado = ' '.join(tokens_filtrados)

    return texto_procesado

def transform_format(val):
    if val == 0:
        return 255
    else:
        return val


def wordcloud_plot(mercadona_hacendado, nombre: str):
  fig = plt.figure(figsize=(20, 12))

  ### preprocesado
  mercadona_hacendado["normalized"] = mercadona_hacendado["tweets"].apply(lambda x:eliminar_espacios(x))
  mercadona_hacendado["normalized"] = mercadona_hacendado["normalized"].apply(lambda x:texto_to_lower(x))
  mercadona_hacendado["normalized"] = mercadona_hacendado["normalized"].apply(lambda x: normalizar_tokens(x))
  mercadona_hacendado["normalized"] = mercadona_hacendado["normalized"].apply(lambda x: lematizar_eliminacion_tokens(x))

    
  txt_cat0 = ",".join(mercadona_hacendado[mercadona_hacendado.search=='mercadona'].normalized.to_list())
  txt_cat1 = ",".join(mercadona_hacendado[mercadona_hacendado.search=='hacendado'].normalized.to_list())

  #mask

  threshold = 128
  mercadona_mask = np.array(Image.open('./data/Silver/logos/Mercadona.png').convert('L'))
  mercadona_mask_resized = np.array(Image.fromarray(mercadona_mask).resize((mercadona_mask.shape[1]*3, mercadona_mask.shape[0]*3)))
  mercadona_mask_binary =  (mercadona_mask_resized > threshold).astype(np.uint8) * 255

  hacendado_mask = np.array(Image.open('./data/Silver/logos/hacendado.png').convert('L'))
  hacendado_mask_resized = np.array(Image.fromarray(hacendado_mask).resize((hacendado_mask.shape[1]*3, hacendado_mask.shape[0]*3)))
  hacendado_mask_binary = (hacendado_mask_resized > threshold).astype(np.uint8) * 255

  #iniciamos Wordcloud

  stopwords_personalizadas = set(es_stopwords)
  palabras_quitar = ['mercadona', 'hacendado']  
  stopwords_personalizadas.update(palabras_quitar)

  wordcloud = WordCloud(background_color="white", max_words=1000, stopwords=stopwords_personalizadas, mask=mercadona_mask_binary,
                      normalize_plurals = True, random_state=2025, contour_width=10, contour_color="forestgreen")


  # Genera el wordcloud
  wordcloud.generate(txt_cat0)
  # Visualizalo en una imagen
  wordcloud.to_image()
  # Mostrar el gráfico con título
  fig = plt.figure(figsize=(20, 12))
  plt.subplot(2, 1, 1)
  plt.imshow(wordcloud, interpolation='bilinear')
  plt.axis("off")  # Ocultar ejes
 # plt.title(f"Clase Negativo  en {nombre}", fontsize=10)
  #plt.show()


  #iniciamos Wordcloud
  wordcloud = WordCloud(background_color="white", max_words=1000, stopwords=stopwords_personalizadas, mask=hacendado_mask_binary,
                      normalize_plurals = True, random_state=2025, contour_width=10, contour_color="black")

    
  # Genera el wordcloud
  wordcloud.generate(txt_cat1)
  wordcloud.to_image()
  # Mostrar el gráfico con título
  #plt.figure(figsize=(16, 10))
  plt.subplot(2, 1, 2)
  plt.imshow(wordcloud, interpolation='bilinear')
  plt.axis("off")  # Ocultar ejes
  #plt.title(f"Clase Positiva  en {nombre}", fontsize=10)  # Aquí colocas tu título
  plt.show()

  return fig


In [ ]:
figura = wordcloud_plot(mercadona_hacendado, "Datos Merged")

In [ ]:
#%matplotlib inline
figura.set_size_inches(10, 6)  
figura.savefig('./data/Gold/graph/wordcloud.png', dpi=300, bbox_inches='tight')